This notebook simulates binary phenotypes by selecting 1000 causal mutations with normally distributed effect sizes (mean=0, variance=1), computing genetic values for each individual based on their genotypes at these causal positions, adding environmental noise to achieve a specified heritability (0.33), and finally converting the continuous liability scores to binary case/control status based on a population prevalence threshold (10% cases). The workflow demonstrates multiple simulation approaches including using a standard model with automatically generated effect sizes, custom user-provided effect sizes, and a standardized genotype matrix option, ultimately producing output files containing individual phenotypes, genetic values, environmental noise components, and the causal mutation effect sizes for downstream analysis.

In [ ]:
import pygrgl
import random

from grg_pheno_sim.model import grg_causal_mutation_model
from grg_pheno_sim.binary_phenotype import sim_binary_phenotypes, sim_binary_phenotypes_custom, sim_binary_phenotypes_standOp


The following command only serves the purpose of converting the VCF zip file into a GRG that will be used for the phenotype simulation. The bash script below will function as expected given the relative path for the source data file is accurate.

In [2]:
%%script bash --out /dev/null
if [ ! -f test-200-samples.grg ]; then
  grg construct -p 10 ../data/test-200-samples.vcf.gz --out-file test-200-samples.grg
fi

In [ ]:
grg_1 = pygrgl.load_immutable_grg("test-200-samples.grg") #loading in a sample grg stored in the same directory

model_type = "normal"
mean = 0
var = 1

model = grg_causal_mutation_model(model_type, mean=mean, var=var)

num_causal = 1000

random_seed = 1

normalize_genetic_values_before_noise = True

heritability = 0.33

population_prevalence = 0.1 #this means 1 in 10 individuals are a case

effect_output_required = True #saves the effect sizes data for each mutation node in a .par file

effect_path = 'univariate_sample_effect_sizes.par'

standardized_output = True

output_path = 'normal_pheno_binary.phen' #define the path to be saved at, this output is saved in the file of this name in the same directory

header=True #set header parameter to true if column names are expected in output file

phenotypes_binary = sim_binary_phenotypes(grg_1, population_prevalence, model, num_causal, random_seed, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability, save_effect_output=effect_output_required, effect_path=effect_path, standardized_output=standardized_output, path=output_path, header=header)


The initial effect sizes are 
     mutation_id  effect_size  causal_mutation_id
0             20    -1.810258                   0
1             28     1.151768                   0
2             62     1.681257                   0
3             76     2.346698                   0
4            119    -0.286668                   0
..           ...          ...                 ...
995        10862    -0.221163                   0
996        10874    -1.136983                   0
997        10879    -0.966133                   0
998        10883    -1.402602                   0
999        10889    -0.483777                   0

[1000 rows x 3 columns]
The genetic values of the individuals are 
     individual_id  genetic_value  causal_mutation_id
0                0     -16.501664                   0
1                1      -2.454348                   0
2                2     -17.303803                   0
3                3       6.641214                   0
4                4      -8.71021

In [4]:
phenotypes_binary

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-0.199939,0.002027,0
1,0,1,0.365408,-0.146442,0
2,0,2,-0.232222,0.200138,0
3,0,3,0.731467,-0.932379,0
4,0,4,0.113635,-0.901380,0
...,...,...,...,...,...
195,0,195,0.968728,-0.931080,0
196,0,196,1.130222,0.279652,1
197,0,197,0.684074,0.155485,0
198,0,198,0.437471,1.258613,1


In [8]:
binary_list = phenotypes_binary["phenotype"]
num_zeros = (binary_list == 0).sum()
num_ones = (binary_list == 1).sum()

print("Number of 0s:", num_zeros)
print("Number of 1s:", num_ones)
print("Population prevalence ratio observed: ", str(num_ones/(num_ones+num_zeros)))


Number of 0s: 187
Number of 1s: 13
Population prevalence ratio observed:  0.065


The observed population prevalence above indicates that our final phenotypes approximately reflect the expected prevalence of 0.1.

Now, we demonstrate a case using default parameter values instead of custom values supplied through the function calls.

In [6]:
grg_1 = pygrgl.load_immutable_grg("test-200-samples.grg") #loading in a sample grg stored in the same directory

heritability = 0.33

population_prevalence = 0.1 #this means 1 in 10 individuals are a case

phenotypes_binary_default = sim_binary_phenotypes(grg_1, population_prevalence, heritability=heritability)


The initial effect sizes are 
     mutation_id  effect_size  causal_mutation_id
0             43    -2.409922                   0
1             49    -0.221536                   0
2             54    -0.286223                   0
3             55     1.130099                   0
4             67     0.052173                   0
..           ...          ...                 ...
995        10798     0.101926                   0
996        10804     1.001745                   0
997        10844    -0.423607                   0
998        10860    -1.222374                   0
999        10888    -1.041588                   0

[1000 rows x 3 columns]
The genetic values of the individuals are 
     individual_id  genetic_value  causal_mutation_id
0                0      -4.913229                   0
1                1       2.868130                   0
2                2      -5.209284                   0
3                3     -18.564395                   0
4                4       3.06880

In [7]:
phenotypes_binary_default

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-0.149286,1.554884,1
1,0,1,0.211137,-1.713347,0
2,0,2,-0.162998,-1.009695,0
3,0,3,-0.781590,0.152136,0
4,0,4,0.220432,-1.205309,0
...,...,...,...,...,...
195,0,195,-0.401219,-1.192500,0
196,0,196,0.046529,1.225813,0
197,0,197,-1.116313,-0.166613,0
198,0,198,-0.393089,0.540116,0


We now demonstrate the use case for a higher population prevalence.

In [8]:
grg_1 = pygrgl.load_immutable_grg("test-200-samples.grg") #loading in a sample grg stored in the same directory

model_type = "normal"
mean = 0
var = 1

model = grg_causal_mutation_model(model_type, mean=mean, var=var)

num_causal = 1000

random_seed = 1

normalize_genetic_values_before_noise = True

heritability = 0.33

population_prevalence_high = 0.5 #this means 1 in 2 individuals are a case

effect_output_required = True #saves the effect sizes data for each mutation node in a .par file

effect_path = 'univariate_sample_effect_sizes.par'

standardized_output = True

output_path = 'normal_pheno_binary_high_prevalence.phen' #define the path to be saved at, this output is saved in the file of this name in the same directory

header=True #set header parameter to true if column names are expected in output file

phenotypes_binary_high_prev = sim_binary_phenotypes(grg_1, population_prevalence_high, model, num_causal, random_seed, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability, save_effect_output=effect_output_required, effect_path=effect_path, standardized_output=standardized_output, path=output_path, header=header)


The initial effect sizes are 
     mutation_id  effect_size  causal_mutation_id
0             20    -1.810258                   0
1             28     1.151768                   0
2             62     1.681257                   0
3             76     2.346698                   0
4            119    -0.286668                   0
..           ...          ...                 ...
995        10862    -0.221163                   0
996        10874    -1.136983                   0
997        10879    -0.966133                   0
998        10883    -1.402602                   0
999        10889    -0.483777                   0

[1000 rows x 3 columns]
The genetic values of the individuals are 
     individual_id  genetic_value  causal_mutation_id
0                0     -16.501664                   0
1                1      -2.454348                   0
2                2     -17.303803                   0
3                3       6.641214                   0
4                4      -8.71021

In [9]:
phenotypes_binary_high_prev

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-0.215496,0.419690,1
1,0,1,0.393839,-0.374444,1
2,0,2,-0.250291,-0.529757,0
3,0,3,0.788380,0.639694,1
4,0,4,0.122476,-0.313749,0
...,...,...,...,...,...
195,0,195,1.044102,-0.651615,1
196,0,196,1.218160,-0.117502,1
197,0,197,0.737300,0.741984,1
198,0,198,0.471509,-0.819500,0


In [10]:
binary_list_high_prev = phenotypes_binary_high_prev["phenotype"]
num_zeros = (binary_list_high_prev == 0).sum()
num_ones = (binary_list_high_prev == 1).sum()

print("Number of 0s:", num_zeros)
print("Number of 1s:", num_ones)
print("Population prevalence ratio observed: ", str(num_ones/(num_ones+num_zeros)))


Number of 0s: 102
Number of 1s: 98
Population prevalence ratio observed:  0.49


We can also use custom effect sizes to compute the binary phenotypes.

In [11]:
random_effects = [random.random() for _ in range(grg_1.num_mutations)] #list input


In [12]:
normalize_genetic_values_before_noise = True

heritability = 0.33

population_prevalence_custom = 0.25

standardized_output = True

output_path = 'custom_pheno_binary.phen' #define the path to be saved at, this output is saved in the file of this name in the same directory

phenotypes_list_binary = sim_binary_phenotypes_custom(grg_1, random_effects, population_prevalence_custom, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability, standardized_output=standardized_output, path=output_path)

The initial effect sizes are 
       mutation_id  effect_size  causal_mutation_id
0                0     0.664109                   0
1                1     0.320532                   0
2                2     0.704756                   0
3                3     0.878410                   0
4                4     0.442122                   0
...            ...          ...                 ...
10888        10888     0.638368                   0
10889        10889     0.421417                   0
10890        10890     0.204887                   0
10891        10891     0.318517                   0
10892        10892     0.415112                   0

[10893 rows x 3 columns]
The genetic values of the individuals are 
     individual_id  genetic_value  causal_mutation_id
0                0    1354.126274                   0
1                1    1389.441769                   0
2                2    1396.873294                   0
3                3    1426.062491                   0
4      

In [13]:
phenotypes_list_binary

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-0.896550,0.781689,0
1,0,1,0.023993,0.068134,0
2,0,2,0.217705,0.145234,0
3,0,3,0.978558,-0.253468,1
4,0,4,-0.633683,-0.698700,0
...,...,...,...,...,...
195,0,195,0.789788,0.702550,1
196,0,196,-0.846185,-0.131998,0
197,0,197,-0.087825,-0.779059,0
198,0,198,-0.361035,-1.382405,0


In [14]:
binary_list_custom = phenotypes_list_binary["phenotype"]
num_zeros = (binary_list_custom == 0).sum()
num_ones = (binary_list_custom == 1).sum()

print("Number of 0s:", num_zeros)
print("Number of 1s:", num_ones)
print("Population prevalence ratio observed: ", str(num_ones/(num_ones+num_zeros)))


Number of 0s: 150
Number of 1s: 50
Population prevalence ratio observed:  0.25


Lastly, all of the previous features are possible with the standardized genotype matrix

In [ ]:
grg_1 = pygrgl.load_immutable_grg("test-200-samples.grg") #loading in a sample grg stored in the same directory

num_causal = 1000

random_seed = 42


heritability = 0.33

population_prevalence = 0.1 #this means 1 in 10 individuals are a case

effect_path = 'univariate_sample_effect_sizes.par'

standardized_output = True

stand_phenotypes_binary = sim_binary_phenotypes(grg_1, population_prevalence, heritability= heritability, num_causal=num_causal, random_seed=random_seed,standardized=True)

# Also with custome effect sizes
random_effects = [random.random() for _ in range(grg_1.num_mutations)] #list input

stand_phenotypes_binary_custom = sim_binary_phenotypes_custom(grg_1,input_effects = random_effects,population_prevalence= population_prevalence, heritability= heritability, random_seed=random_seed,standardized=True)

#####  RUN 1  #####
The initial effect sizes are 
     mutation_id  effect_size  causal_mutation_id
0             43    -0.043778                   0
1             49    -0.004024                   0
2             54    -0.005199                   0
3             55     0.020529                   0
4             67     0.000948                   0
..           ...          ...                 ...
995        10798     0.001852                   0
996        10804     0.018198                   0
997        10844    -0.007695                   0
998        10860    -0.022206                   0
999        10888    -0.018921                   0

[1000 rows x 3 columns]
The genetic values of the individuals are 
     individual_id  genetic_value  causal_mutation_id
0                0      -0.698857                   0
1                1       0.285342                   0
2                2       0.270383                   0
3                3      -0.418744                   0
4           